### Recommendations using surprise library models:

This notebook uses the surprise library to build a model using surprise library and generate ratings for the test data. \\
**Models:** SVD, Baseline estimation, item-item based KNN collaborative filtering etc are explored. Hyperparameter tuning for the models is performed. \\
**Evaluation:** The test data ratings are stored for further evaluation such as RMSE and MAE. The recommendations are evaluated using precision@5, recall@5, NDCG and overall accuracy. 

In [ ]:
from surprise import SVD, BaselineOnly, SVDpp, NMF, SlopeOne, CoClustering, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore
from surprise import accuracy
from surprise.model_selection import train_test_split

In [ ]:
import math
from collections import defaultdict
import csv
from sklearn.metrics import ndcg_score
import numpy as np
import pandas as pd
import time

In [ ]:
def convert_traintest_dataframe_forsurprise(training_dataframe, testing_dataframe):
    reader = Reader(rating_scale=(0, 5))
    trainset = Dataset.load_from_df(training_dataframe[['userId', 'movieId', 'rating']], reader)
    testset = Dataset.load_from_df(testing_dataframe[['userId', 'movieId', 'rating']], reader)
    trainset = trainset.construct_trainset(trainset.raw_ratings)
    testset = testset.construct_testset(testset.raw_ratings)
    return trainset, testset

In [ ]:
file_path_train = 'training_data.csv'
file_path_test = 'testing_data.csv'
traindf = pd.read_csv(file_path_train)
testdf = pd.read_csv(file_path_test)
trainset, testset = convert_traintest_dataframe_forsurprise(traindf, testdf)

In [ ]:
def get_top_n(predictions, n):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    org_ratings = defaultdict(list)

    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
        org_ratings[uid].append((iid, true_r))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n, org_ratings

In [ ]:
def dcg_at_k(scores):
    return scores[0] + sum(sc/math.log(ind, 2) for sc, ind in zip(scores[1:], range(2, len(scores) + 1)))

def ndcg_at_k(scores):
    idcg = dcg_at_k(sorted(scores, reverse=True))
    return (dcg_at_k(scores)/idcg) if idcg > 0.0 else 0.0

In [ ]:
def precision_recall_at_k(predictions, k=5, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    precision = (sum(prec for prec in precisions.values()) / len(precisions))
    recall = (sum(rec for rec in recalls.values()) / len(recalls))

    return precision, recall

In [ ]:
def recommendation(algo, trainset, testset):
  # Train the algorithm on the trainset, and predict ratings for the testset
  start_fit = time.time()
  algo.fit(trainset)
  end_fit = time.time()
  fit_time = end_fit - start_fit

  # Predictions on testing set
  start_test = time.time()
  test_predictions = algo.test(testset)
  end_test = time.time()
  test_time = end_test - start_test

  test_rmse = accuracy.rmse(test_predictions)
  test_mae = accuracy.mae(test_predictions)

  top_n, org_ratings = get_top_n(test_predictions, 5)

  precision, recall = precision_recall_at_k(test_predictions)

  f_measure = (2*precision*recall)/(precision+recall)

  ndcg_scores = dict()
  for uid, user_ratings in top_n.items():
    scores = []
    for iid, est_r in user_ratings:
        iid_found = False
        org_user_ratings = org_ratings[uid]
        for i, r in org_user_ratings:
            if iid == i:
                scores.append(r)
                iid_found = True
                break
        if not iid_found:
            scores.append(0)
    ndcg_scores[uid] = ndcg_at_k(scores)
  ndcg_score = sum(ndcg for ndcg in ndcg_scores.values())/len(ndcg_scores)

  return (test_rmse, test_mae, fit_time, test_time, precision, recall, f_measure, ndcg_score,test_predictions)

#### Basic algorithm (Baseline approach):

In [ ]:
# basic collaborative filtering algorithm taking into account a baseline rating.
sim_options = {'name': 'pearson_baseline',
               'user_based': False  # compute  similarities between items
               }
algo = KNNBaseline(sim_options=sim_options)

results = recommendation(algo,trainset,testset)
print(results[0])
print(results[1])
print(results[2])
print(results[3])
print(results[4])
print(results[5])
print(results[6])
print(results[7])

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8527
MAE:  0.6482
0.8527048407985288
0.6481831050363938
11.110082864761353
6.931541919708252
0.8311748633879806
0.4131693896380987
0.5519630281626648
0.9631023306515691


In [ ]:
algo = CoClustering(2,5,50)

test_rmse, test_mae, test_predictions, fit_time, test_time, precision, recall, f_measure, ndcg_score = recommendation(algo,trainset,testset)
print(test_rmse)
print(test_mae)
print(fit_time)
print(test_time)
print(precision)
print(recall)
print(f_measure)
print(ndcg_score)

RMSE: 0.9464
MAE:  0.7297
0.946409332531524
0.7297237363608594
4.6282360553741455
0.09603595733642578
0.7793689825657039
0.5126558500741573
0.6184835742859471
1.0186988181654881


In [ ]:
surprise_df = pd.DataFrame(columns= ['Algorithm', 'test_rmse', 'test_mae', 'fit_time', 'test_time', 'Precision', 'Recall', 'F-measure', 'NDCG'])

In [ ]:
# Iterate over all algorithms
for algorithm in [KNNBasic(), SVD(), SVDpp(), SlopeOne(), NMF(), KNNBaseline(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    results = recommendation(algorithm,trainset,testset) 
    
    name =str(algorithm).split(' ')[0].split('.')[-1]
    print("Algorithm:", name)
    df = pd.DataFrame([[name, results[0], results[1], results[2], results[3], results[4], results[5], results[6], results[7]]], columns= ['Algorithm', 'test_rmse', 'test_mae', 'fit_time', 'test_time', 'Precision', 'Recall', 'F-measure', 'NDCG'])
    surprise_df = pd.concat([df, surprise_df], ignore_index=True)
surprise_df.sort_values(by='test_rmse', ascending=False) 

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9508
MAE:  0.7267
Algorithm: KNNBasic
RMSE: 0.8794
MAE:  0.6739
Algorithm: SVD
RMSE: 0.8691
MAE:  0.6641
Algorithm: SVDpp
RMSE: 0.9056
MAE:  0.6877
Algorithm: SlopeOne
RMSE: 0.9291
MAE:  0.7095
Algorithm: NMF
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8763
MAE:  0.6660
Algorithm: KNNBaseline
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9000
MAE:  0.6837
Algorithm: KNNWithMeans
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8992
MAE:  0.6786
Algorithm: KNNWithZScore
Estimating biases using als...
RMSE: 0.8735
MAE:  0.6718
Algorithm: BaselineOnly
RMSE: 0.9522
MAE:  0.7333
Algorithm: CoClustering


,Algorithm,test_rmse,test_mae,fit_time,test_time,Precision,Recall,F-measure,NDCG
0,CoClustering,0.952208,0.733264,1.825201,0.173414,0.782623,0.380981,0.512484,0.955748
9,KNNBasic,0.950772,0.726653,0.098789,1.389246,0.783880,0.421535,0.548247,0.958676
5,NMF,0.929142,0.709469,5.150043,0.204061,0.779290,0.380745,0.511555,0.954884
6,SlopeOne,0.905645,0.687692,4.759230,5.592333,0.807541,0.396525,0.531881,0.955546
3,KNNWithMeans,0.900034,0.683693,0.109672,1.540596,0.800956,0.387139,0.521981,0.952709
2,KNNWithZScore,0.899184,0.678623,0.146574,1.795092,0.794809,0.391525,0.524620,0.954195
8,SVD,0.879436,0.673949,4.609977,0.125070,0.803306,0.385542,0.521022,0.956596
4,KNNBaseline,0.876293,0.665992,0.210110,1.936552,0.796421,0.415881,0.546425,0.956228
1,BaselineOnly,0.873457,0.671770,0.139662,0.092511,0.807432,0.400350,0.535288,0.959085
7,SVDpp,0.869119,0.664051,480.697082,7.997302,0.817842,0.397884,0.535328,0.960317


In [ ]:
surprise_df.sort_values(by='test_rmse') 

,Algorithm,test_rmse,test_mae,fit_time,test_time,Precision,Recall,F-measure,NDCG
7,SVDpp,0.869119,0.664051,480.697082,7.997302,0.817842,0.397884,0.535328,0.960317
1,BaselineOnly,0.873457,0.671770,0.139662,0.092511,0.807432,0.400350,0.535288,0.959085
4,KNNBaseline,0.876293,0.665992,0.210110,1.936552,0.796421,0.415881,0.546425,0.956228
8,SVD,0.879436,0.673949,4.609977,0.125070,0.803306,0.385542,0.521022,0.956596
2,KNNWithZScore,0.899184,0.678623,0.146574,1.795092,0.794809,0.391525,0.524620,0.954195
3,KNNWithMeans,0.900034,0.683693,0.109672,1.540596,0.800956,0.387139,0.521981,0.952709
6,SlopeOne,0.905645,0.687692,4.759230,5.592333,0.807541,0.396525,0.531881,0.955546
5,NMF,0.929142,0.709469,5.150043,0.204061,0.779290,0.380745,0.511555,0.954884
9,KNNBasic,0.950772,0.726653,0.098789,1.389246,0.783880,0.421535,0.548247,0.958676
0,CoClustering,0.952208,0.733264,1.825201,0.173414,0.782623,0.380981,0.512484,0.955748


In [ ]:
surprise_df.sort_values(by='F-measure', ascending=False) 

,Algorithm,test_rmse,test_mae,fit_time,test_time,Precision,Recall,F-measure,NDCG
9,KNNBasic,0.950772,0.726653,0.098789,1.389246,0.783880,0.421535,0.548247,0.958676
4,KNNBaseline,0.876293,0.665992,0.210110,1.936552,0.796421,0.415881,0.546425,0.956228
7,SVDpp,0.869119,0.664051,480.697082,7.997302,0.817842,0.397884,0.535328,0.960317
1,BaselineOnly,0.873457,0.671770,0.139662,0.092511,0.807432,0.400350,0.535288,0.959085
6,SlopeOne,0.905645,0.687692,4.759230,5.592333,0.807541,0.396525,0.531881,0.955546
2,KNNWithZScore,0.899184,0.678623,0.146574,1.795092,0.794809,0.391525,0.524620,0.954195
3,KNNWithMeans,0.900034,0.683693,0.109672,1.540596,0.800956,0.387139,0.521981,0.952709
8,SVD,0.879436,0.673949,4.609977,0.125070,0.803306,0.385542,0.521022,0.956596
0,CoClustering,0.952208,0.733264,1.825201,0.173414,0.782623,0.380981,0.512484,0.955748
5,NMF,0.929142,0.709469,5.150043,0.204061,0.779290,0.380745,0.511555,0.954884


In [ ]:
surprise_df.sort_values(by='NDCG', ascending=False)

,Algorithm,test_rmse,test_mae,fit_time,test_time,Precision,Recall,F-measure,NDCG
7,SVDpp,0.869119,0.664051,480.697082,7.997302,0.817842,0.397884,0.535328,0.960317
1,BaselineOnly,0.873457,0.671770,0.139662,0.092511,0.807432,0.400350,0.535288,0.959085
9,KNNBasic,0.950772,0.726653,0.098789,1.389246,0.783880,0.421535,0.548247,0.958676
8,SVD,0.879436,0.673949,4.609977,0.125070,0.803306,0.385542,0.521022,0.956596
4,KNNBaseline,0.876293,0.665992,0.210110,1.936552,0.796421,0.415881,0.546425,0.956228
0,CoClustering,0.952208,0.733264,1.825201,0.173414,0.782623,0.380981,0.512484,0.955748
6,SlopeOne,0.905645,0.687692,4.759230,5.592333,0.807541,0.396525,0.531881,0.955546
5,NMF,0.929142,0.709469,5.150043,0.204061,0.779290,0.380745,0.511555,0.954884
2,KNNWithZScore,0.899184,0.678623,0.146574,1.795092,0.794809,0.391525,0.524620,0.954195
3,KNNWithMeans,0.900034,0.683693,0.109672,1.540596,0.800956,0.387139,0.521981,0.952709


In [ ]:
sim_options = {'name': 'pearson_baseline',
               'user_based': False  # compute  similarities between items
               }
algo = KNNBaseline(sim_options=sim_options)

results = recommendation(algo,trainset,testset)
df = pd.DataFrame([['KNNBaseline (pearson_baseline)', results[0], results[1], results[2], results[3], results[4], results[5], results[6], results[7]]], columns= ['Algorithm', 'test_rmse', 'test_mae', 'fit_time', 'test_time', 'Precision', 'Recall', 'F-measure', 'NDCG'])
surprise_df = pd.concat([df, surprise_df], ignore_index=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8527
MAE:  0.6482


In [ ]:
surprise_df.head()

,Algorithm,test_rmse,test_mae,fit_time,test_time,Precision,Recall,F-measure,NDCG
0,KNNBaseline (pearson_baseline,0.852705,0.648183,9.183264,6.478208,0.831175,0.413169,0.551963,0.963102
1,CoClustering,0.952208,0.733264,1.825201,0.173414,0.782623,0.380981,0.512484,0.955748
2,BaselineOnly,0.873457,0.671770,0.139662,0.092511,0.807432,0.400350,0.535288,0.959085
3,KNNWithZScore,0.899184,0.678623,0.146574,1.795092,0.794809,0.391525,0.524620,0.954195
4,KNNWithMeans,0.900034,0.683693,0.109672,1.540596,0.800956,0.387139,0.521981,0.952709


In [ ]:
surprise_df.to_csv('Surprise_results.csv')

In [ ]:
surprise_df.sort_values(by='test_rmse') 

,Algorithm,test_rmse,test_mae,fit_time,test_time,Precision,Recall,F-measure,NDCG
0,KNNBaseline (pearson_baseline,0.852705,0.648183,9.183264,6.478208,0.831175,0.413169,0.551963,0.963102
8,SVDpp,0.869119,0.664051,480.697082,7.997302,0.817842,0.397884,0.535328,0.960317
2,BaselineOnly,0.873457,0.671770,0.139662,0.092511,0.807432,0.400350,0.535288,0.959085
5,KNNBaseline,0.876293,0.665992,0.210110,1.936552,0.796421,0.415881,0.546425,0.956228
9,SVD,0.879436,0.673949,4.609977,0.125070,0.803306,0.385542,0.521022,0.956596
3,KNNWithZScore,0.899184,0.678623,0.146574,1.795092,0.794809,0.391525,0.524620,0.954195
4,KNNWithMeans,0.900034,0.683693,0.109672,1.540596,0.800956,0.387139,0.521981,0.952709
7,SlopeOne,0.905645,0.687692,4.759230,5.592333,0.807541,0.396525,0.531881,0.955546
6,NMF,0.929142,0.709469,5.150043,0.204061,0.779290,0.380745,0.511555,0.954884
10,KNNBasic,0.950772,0.726653,0.098789,1.389246,0.783880,0.421535,0.548247,0.958676


In [ ]:
surprise_df.sort_values(by='F-measure', ascending=False) 

,Algorithm,test_rmse,test_mae,fit_time,test_time,Precision,Recall,F-measure,NDCG
0,KNNBaseline (pearson_baseline,0.852705,0.648183,9.183264,6.478208,0.831175,0.413169,0.551963,0.963102
10,KNNBasic,0.950772,0.726653,0.098789,1.389246,0.783880,0.421535,0.548247,0.958676
5,KNNBaseline,0.876293,0.665992,0.210110,1.936552,0.796421,0.415881,0.546425,0.956228
8,SVDpp,0.869119,0.664051,480.697082,7.997302,0.817842,0.397884,0.535328,0.960317
2,BaselineOnly,0.873457,0.671770,0.139662,0.092511,0.807432,0.400350,0.535288,0.959085
7,SlopeOne,0.905645,0.687692,4.759230,5.592333,0.807541,0.396525,0.531881,0.955546
3,KNNWithZScore,0.899184,0.678623,0.146574,1.795092,0.794809,0.391525,0.524620,0.954195
4,KNNWithMeans,0.900034,0.683693,0.109672,1.540596,0.800956,0.387139,0.521981,0.952709
9,SVD,0.879436,0.673949,4.609977,0.125070,0.803306,0.385542,0.521022,0.956596
1,CoClustering,0.952208,0.733264,1.825201,0.173414,0.782623,0.380981,0.512484,0.955748


In [ ]:
surprise_df.sort_values(by='NDCG', ascending=False)

,Algorithm,test_rmse,test_mae,fit_time,test_time,Precision,Recall,F-measure,NDCG
0,KNNBaseline (pearson_baseline,0.852705,0.648183,9.183264,6.478208,0.831175,0.413169,0.551963,0.963102
8,SVDpp,0.869119,0.664051,480.697082,7.997302,0.817842,0.397884,0.535328,0.960317
2,BaselineOnly,0.873457,0.671770,0.139662,0.092511,0.807432,0.400350,0.535288,0.959085
10,KNNBasic,0.950772,0.726653,0.098789,1.389246,0.783880,0.421535,0.548247,0.958676
9,SVD,0.879436,0.673949,4.609977,0.125070,0.803306,0.385542,0.521022,0.956596
5,KNNBaseline,0.876293,0.665992,0.210110,1.936552,0.796421,0.415881,0.546425,0.956228
1,CoClustering,0.952208,0.733264,1.825201,0.173414,0.782623,0.380981,0.512484,0.955748
7,SlopeOne,0.905645,0.687692,4.759230,5.592333,0.807541,0.396525,0.531881,0.955546
6,NMF,0.929142,0.709469,5.150043,0.204061,0.779290,0.380745,0.511555,0.954884
3,KNNWithZScore,0.899184,0.678623,0.146574,1.795092,0.794809,0.391525,0.524620,0.954195
